In [1]:
import os
import numpy as np
import cv2
import pandas as pd
import torch

In [12]:
import torch.utils.data

In [2]:
feature_table_path = "../feature/feature_table.pkl"
feature_table = pd.read_pickle(feature_table_path)

TRAIN_FOLDER_NAME = "../train_img"

In [4]:
def table_division(feature_table):
    devided_df = []
    for label, df in feature_table.groupby("label"):
        devided_df.append(df)

    return devided_df

def over_search_img(df,img_num):
    images = list(df["img"][:img_num])

    return images

In [3]:
def create_train_dataset(feature_table, TRAIN_FOLDER_NAME,image_set):
    train_dataset = []
    for image in image_set:
        label = feature_table[feature_table.img == image].label.values[0]
        filepath = os.path.join(TRAIN_FOLDER_NAME, str(label), image)
        train_dataset.append((filepath, np.int32(label)))

    return train_dataset

def dataset_conversion(dataset):
        img_data = []
        label_data = []

        for filepath, label in dataset:
            img = cv2.imread(filepath, 1)
            img = cv2.resize(img,(28,28))
            img_data.append(img)
            label_data.append(label)

        img_data = np.array(img_data).astype(np.float32)
        label_data = np.array(label_data).astype(np.int32)

        return img_data, label_data

In [5]:
img_num_by_class = 300
devide_df = table_division(feature_table)

image_set = []
for df in devide_df:
    image_set += over_search_img(df, img_num_by_class)

In [7]:
train_dataset = create_train_dataset(feature_table, TRAIN_FOLDER_NAME, image_set)
img_data, label_data = dataset_conversion(train_dataset)

train_data = []
for x_train, y_train in zip(img_data, label_data):
    train_data.append((x_train, y_train))

In [11]:
dataloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)